In [21]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

## Carga CFI en Mercado COLOCA para Calculo Comisiones
### 1.- Query saldos de custodias:  BCS Server AI
USE BCS
GO

select cod_per Periodo,b.rut_cli Rut,b.sec_rut_cli Secuencia,b.nombre_cli Cliente,b.cod_agente,a.nemo Nemo,sum(CAN_ACTUAL + cAN_GARANTIA) AS CANTIDAD_CUOTAS
 from TBCTCIRV a 
inner join TBPLFICL b on a.rut_cli=b.rut_cli and a.sec_rut_cli=b.sec_rut_cli where cod_per='2023-06' and a.mercado='CF' and COD_BOVEDA='COR' group by cod_per,a.nemo,b.rut_cli,b.sec_rut_cli,b.nombre_cli,b.cod_agente
## 1.- ejecutar query de saldo de custodia al mes anterior
### 2.- Crear Archivo de carga,  con dos pestañas segun formato anterior
### 3.- Revisar Nemos sin Custodia 
### 4.- actualizar fecha periodo para archivo de carga


In [22]:

#archivo= 'Cierre Mensual CFI ENE23.xlsx'
archivo= 'Cierre Mensual CFI - Edicion - Postproceso.xlsx'
patrimonio= pd.read_excel(archivo, sheet_name='custodia')
Facturacion= pd.read_excel(archivo, sheet_name='Consolidado2', header=0).dropna()#, index_col=None, header=None)

#Facturacion.where(Facturacion[1]>0)
#Facturacion.query('Fact>0')
patrimonio['Nemo']=patrimonio['Nemo'].str.strip()
Facturacion['Nemo']=Facturacion['Nemo'].str.strip()
Facturacion.tail(10)

,Nemo,Fact,emisor
13,CFIBTGDEA,13805.00,BTG
16,CFICGSCH-A /CFICGSCH-I,5955.00,Compass
17,CFICGGTA-E,4264.00,Compass
20,CFI-HMCBGB,13188.00,HMC
21,CFIPD7BP-E,4692156.00,HMC
22,CFIHMCDPGP,6901.00,HMC
23,CFI-HMCDGD,410172.00,HMC
24,CFIHMCRGPB,685974.00,HMC
28,CFIHMCREPA,4474.00,HMC
29,CFI-HMCUSD / CFIHMCUSBP,114842.00,HMC


In [23]:
set(Facturacion.emisor.to_list())



{'BTG', 'Compass', 'HMC'}

In [24]:
patrimonio= patrimonio.merge(Facturacion, left_on='Nemo', right_on='Nemo')#.to_excel('NuevaCargaFip.xlsx')

#Eliminar comisiones A fondos HMC
HMC= {'RUBIX': [9500], 
        'HMC':[7000],
        'ALTAVISTA': [7200], 'ALTAVISTA2': [7201],'B&A GESTION DE ACTIVOS': [9600]}
#Eliminar comisiones A fondos Sartor
Sartor ={'RUBIX': [9500],
       'BD INVESTMENTS': [7696],'BD INVESTMENTS2': [7697],
        'ASESORÍAS E INVERSIONES C&C SPA':[7699]}


ds=pd.DataFrame(Sartor).T
ds2= pd.DataFrame(HMC).T



In [25]:
ds

,0
RUBIX,9500
BD INVESTMENTS,7696
BD INVESTMENTS2,7697
ASESORÍAS E INVERSIONES C&C SPA,7699


In [26]:
patrimonio

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-04,5138644-2,0,ALIAGA BRAVO JORGE ALFREDO ...,9500,CFI-HMCBGB,9205.00,13188.00,HMC
1,2025-04,6617829-3,0,SERRANO ALDERETE IVAN ERDMAN ...,9500,CFI-HMCBGB,9063.00,13188.00,HMC
2,2025-04,7317272-1,0,ORTIZ DONAIRE ANGELA ...,9500,CFI-HMCBGB,2187.00,13188.00,HMC
3,2025-04,7622428-5,1,PEREZ VALDES ANDRES ...,9500,CFI-HMCBGB,2107.00,13188.00,HMC
4,2025-04,8149055-4,0,VILLALOBOS CARVAJAL RICARDO DARÍO ...,9500,CFI-HMCBGB,10151.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
2380,2025-04,25880866-5,0,Rolin FOURCROY BAPTISTE LOUIS...,9500,CFISCCH,31.00,63475.00,BTG
2381,2025-04,76104590-3,0,INVERSIONES Y ASESORIAS ALTOS DE SANTA FE LTDA...,3308,CFISCCH,4.00,63475.00,BTG
2382,2025-04,77475773-2,0,SOCIEDAD DE PRESTACIONES MÉDICAS AGUSTINAS SPA...,9500,CFISCCH,102.00,63475.00,BTG
2383,2025-04,77517228-2,0,SOCIEDAD MEDICA E INVERSIONES GALLEGOS MORAN S...,9500,CFISCCH,5.00,63475.00,BTG


In [27]:
Nemo=Facturacion['Nemo'].to_list()
Nemo
#Facturacion['Fact'].sum()


['CFISCCH',
 'CFIBTGRE-E',
 'CFIBPDCCHA',
 'CFIBTGDAPA',
 'CFIBTGPLAA',
 'CFIBTGCYFA',
 'CFIBTGRCA',
 'CFICELDI1',
 'CFIBTGFGIA',
 'CFIBTGRRH',
 'CFICELDI2',
 'CFIBTGRRA',
 'CFIBCCHIGA',
 'CFIBTGDEA',
 'CFICGSCH-A /CFICGSCH-I',
 'CFICGGTA-E',
 'CFI-HMCBGB',
 'CFIPD7BP-E',
 'CFIHMCDPGP',
 'CFI-HMCDGD',
 'CFIHMCRGPB',
 'CFIHMCREPA',
 'CFI-HMCUSD / CFIHMCUSBP']

In [28]:
#i='CFITACTI-A'
#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio['prorratra']=patrimonio['CANTIDAD_CUOTAS']*202789.075630252/patrimonio['cuotasNVS']
#print(patrimonio.query('Nemo=="CFISCCH"')) # si da cero es por problema de datos, losnemos tienen espacios

#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio

In [29]:
#No paga

No_pago_Sartor =[9500,7696,7697,7699]
No_pago_HMC =  [6800, 7000, 7200, 7201, 7600, 7696, 7697, 9500, 9600]  #[9500,7000, 7200,7201,9600]
No_pago_Credicorp = [9500, 7600, 7697, 7696]
No_pago_BCI =[9500]
No_pago_Toesca=[6800]
No_pago_MBI =[9500, 7600, 7697, 7696]
No_pago_BTG =[9500]
No_pago_Compass=[9500, 7600, 7697, 7696]



test =patrimonio
for index in range(test.shape[0]):
    if test.iloc[index,8]=='HMC' and test.iloc[index,4] in No_pago_HMC:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Sartor' and test.iloc[index,4] in No_pago_Sartor:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
# nuevo 2025-03
# no pago credicorp

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Credicorp' and test.iloc[index,4] in No_pago_Credicorp:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
#No pago BCI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BCI' and test.iloc[index,4] in No_pago_BCI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

#No pago Toesca
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Toesca' and test.iloc[index,4] in No_pago_Toesca:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago MBI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='MBI' and test.iloc[index,4] in No_pago_MBI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago BTG

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BTG' and test.iloc[index,4] in No_pago_BTG:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Compass' and test.iloc[index,4] in No_pago_Compass:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

In [30]:
test.query('Cliente=="No Pago"')

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-04,5138644-2,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
1,2025-04,6617829-3,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
2,2025-04,7317272-1,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
3,2025-04,7622428-5,1,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
4,2025-04,8149055-4,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
2379,2025-04,24732559-K,1,No Pago,9500,CFISCCH,0.00,63475.00,BTG
2380,2025-04,25880866-5,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG
2382,2025-04,77475773-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG
2383,2025-04,77517228-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG


In [31]:

print(index)

2384


In [32]:
test

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-04,5138644-2,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
1,2025-04,6617829-3,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
2,2025-04,7317272-1,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
3,2025-04,7622428-5,1,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
4,2025-04,8149055-4,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
2380,2025-04,25880866-5,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG
2381,2025-04,76104590-3,0,INVERSIONES Y ASESORIAS ALTOS DE SANTA FE LTDA...,3308,CFISCCH,4.00,63475.00,BTG
2382,2025-04,77475773-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG
2383,2025-04,77517228-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG


In [33]:

Nemo=Facturacion.query('Fact>0').copy()
Nemo=Nemo['Nemo'].to_list()
print(Nemo)
print(len(Nemo))


['CFISCCH', 'CFIBTGRE-E', 'CFIBPDCCHA', 'CFIBTGPLAA', 'CFIBTGCYFA', 'CFIBTGRCA', 'CFICELDI1', 'CFIBTGFGIA', 'CFIBTGRRH', 'CFICELDI2', 'CFIBTGRRA', 'CFIBCCHIGA', 'CFIBTGDEA', 'CFICGSCH-A /CFICGSCH-I', 'CFICGGTA-E', 'CFI-HMCBGB', 'CFIPD7BP-E', 'CFIHMCDPGP', 'CFI-HMCDGD', 'CFIHMCRGPB', 'CFIHMCREPA', 'CFI-HMCUSD / CFIHMCUSBP']
22


In [34]:
for i in Nemo:
    if patrimonio.query('Nemo=="'+i+'"').empty :
        print(i)

CFIBTGRE-E
CFICGSCH-A /CFICGSCH-I
CFICGGTA-E
CFI-HMCUSD / CFIHMCUSBP


In [35]:
inic=Nemo[0]
print(inic)
Carga=patrimonio.query('Nemo=="'+inic+'"').copy()

Carga['cuotasNVS']=Carga.query('Nemo=="'+inic+'"')['CANTIDAD_CUOTAS'].sum()
Carga['monto']=Carga['CANTIDAD_CUOTAS']
Carga['acciones']=Carga['CANTIDAD_CUOTAS']
print('1')
Carga['comision']=Carga['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+inic+'"')['Fact'].iloc[0])/Carga['cuotasNVS']
Carga['porcentaje']=Carga['CANTIDAD_CUOTAS']/Carga['cuotasNVS']
print('1')
Carga['emisor'] = Facturacion.query('Nemo=="'+inic+'"').iat[0,2]
Carga['mercado'] = 'RV'
Carga['FECHA'] = '31-05-2025'

CFISCCH
1
1


In [36]:
#Carga=patrimonio.query('Nemo=="CFISCCH"').copy()
Carga

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA
2341,2025-04,8427228-0,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2342,2025-04,9497244-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2343,2025-04,9913936-6,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2344,2025-04,9979324-4,1,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2345,2025-04,9984895-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2346,2025-04,12246880-1,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2347,2025-04,12562443-K,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2348,2025-04,12699269-6,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2349,2025-04,12856206-0,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2350,2025-04,12989521-7,2,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025


In [37]:
for i in Nemo[1:]:
    if patrimonio.query('Nemo=="'+i+'"').empty==False:
        dfpaso=patrimonio.query('Nemo=="'+i+'"').copy()
        dfpaso['cuotasNVS']=dfpaso.query('Nemo=="'+i+'"')['CANTIDAD_CUOTAS'].sum()
        dfpaso['monto']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['acciones']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['comision']=dfpaso['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+i+'"')['Fact'].iloc[0])/dfpaso['cuotasNVS']
        dfpaso['porcentaje']=dfpaso['CANTIDAD_CUOTAS']/dfpaso['cuotasNVS']
        dfpaso['emisor'] = Facturacion.query('Nemo=="'+i+'"').iat[0,2]
        dfpaso['mercado'] = 'RV'
        dfpaso['FECHA'] = '31-05-2025'
        Carga=pd.concat([Carga,dfpaso], sort=False)

###### Carga.keys()

In [38]:
Carga.reset_index()

,index,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA
0,2341,2025-04,8427228-0,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
1,2342,2025-04,9497244-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2,2343,2025-04,9913936-6,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
3,2344,2025-04,9979324-4,1,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
4,2345,2025-04,9984895-2,0,No Pago,9500,CFISCCH,0.00,63475.00,BTG,5.00,0.00,0.00,0.00,0.00,RV,31-05-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,1839,2025-04,77813450-0,0,No Pago,7000,CFIHMCREPA,0.00,4474.00,HMC,1503.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2379,1840,2025-04,77972127-2,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,1503.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2380,1841,2025-04,78010895-9,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,1503.00,0.00,0.00,0.00,0.00,RV,31-05-2025
2381,1842,2025-04,78097270-K,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,1503.00,0.00,0.00,0.00,0.00,RV,31-05-2025


In [39]:
CargaCSV=Carga[['Rut','monto','acciones','porcentaje','comision','emisor' ,'Nemo','mercado','FECHA']]

In [40]:
CargaCSV.to_excel('CargafipLAST0425.xlsx')

In [41]:
CargaCSV.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2383 entries, 2341 to 1843
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rut         2383 non-null   object 
 1   monto       2383 non-null   float64
 2   acciones    2383 non-null   float64
 3   porcentaje  2382 non-null   float64
 4   comision    2382 non-null   float64
 5   emisor      2383 non-null   object 
 6   Nemo        2383 non-null   object 
 7   mercado     2383 non-null   object 
 8   FECHA       2383 non-null   object 
dtypes: float64(4), object(5)
memory usage: 186.2+ KB
